# Cloud Carbon Coefficients

This notebook calculates the coefficients used in [Cloud Carbon Footprint](https://www.cloudcarbonfootprint.org/), an application that estimates the energy (kilowatt hours) and carbon emissions (metric tons CO2e) of public cloud provider utilization.

#### Imports

In [1]:
%pip install -r requirements.txt

import csv
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


## Processor types

Processors are grouped into types by each vendor e.g. Intel Broadwell CPUs. Cloud providers provide the CPU types for each of their instance types, but not the precise CPU details. As such, we calculate the average wattage by processor type.

In [2]:
# Loads a CSV file then returns each row appended to an array
def load_append_data(file_name):
    with open(f'data/{file_name}', 'r') as csvfile:
        reader = csv.reader(csvfile)

        data = []
        for row in reader:
            data.append(row[0])
        
        return data

cpus_intel_sandybridge = load_append_data('intel-sandybridge.csv')
cpus_intel_ivybridge = load_append_data('intel-ivybridge.csv')
cpus_intel_haswell = load_append_data('intel-haswell.csv')
cpus_intel_broadwell = load_append_data('intel-broadwell.csv')
cpus_intel_skylake = load_append_data('intel-skylake.csv')
cpus_intel_cascadelake = load_append_data('intel-cascadelake.csv')
cpus_intel_coffeelake = load_append_data('intel-coffeelake.csv')

## Processor lists

Now we know which processors are in which type, we can group all the tested servers by their CPU type to calculate: average idle watts, average watts at 100% utilization, average GB/chip.

In [3]:
# Load all servers from SPECpower results CSV
servers = pd.read_csv('data/SPECpower-full-results.csv', na_values=['NC'])

### Intel: Sandy Bridge

In [4]:
servers_intel_sandybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_sandybridge))]

intel_sandybridge = {}
intel_sandybridge['idle_watts'] = (servers_intel_sandybridge['avg. watts @ active idle'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['100% watts'] = (servers_intel_sandybridge['avg. watts @ 100%'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['GB/Chip'] = (servers_intel_sandybridge['Total Memory (GB)'] / servers_intel_sandybridge['Chips']).mean()
intel_sandybridge

{'idle_watts': 2.212455527435446,
 '100% watts': 8.460144657004527,
 'GB/Chip': 16.25294117647059}

### Intel: Ivy Bridge

In [5]:
servers_intel_ivybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_ivybridge))]

intel_ivybridge = {}
intel_ivybridge['idle_watts'] = (servers_intel_ivybridge['avg. watts @ active idle'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['100% watts'] = (servers_intel_ivybridge['avg. watts @ 100%'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['GB/Chip'] = (servers_intel_ivybridge['Total Memory (GB)'] / servers_intel_ivybridge['Chips']).mean()
intel_ivybridge

{'idle_watts': 4.107881944444444,
 '100% watts': 9.415972222222223,
 'GB/Chip': 16.32}

### Intel: Haswell

In [6]:
servers_intel_haswell = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_haswell))]

intel_haswell = {}
intel_haswell['idle_watts'] = (servers_intel_haswell['avg. watts @ active idle'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['100% watts'] = (servers_intel_haswell['avg. watts @ 100%'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['GB/Chip'] = (servers_intel_haswell['Total Memory (GB)'] / servers_intel_haswell['Chips']).mean()
intel_haswell

{'idle_watts': 0.9512500000000002,
 '100% watts': 4.694826388888889,
 'GB/Chip': 28.16}

### Intel: Broadwell

In [7]:
servers_intel_broadwell = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_broadwell))]

intel_broadwell = {}
intel_broadwell['idle_watts'] = (servers_intel_broadwell['avg. watts @ active idle'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['100% watts'] = (servers_intel_broadwell['avg. watts @ 100%'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['GB/Chip'] = (servers_intel_broadwell['Total Memory (GB)'] / servers_intel_broadwell['Chips']).mean()
intel_broadwell

{'idle_watts': 0.7128342245989304,
 '100% watts': 3.6853275401069516,
 'GB/Chip': 69.6470588235294}

### Intel: Sky Lake

In [8]:
servers_intel_skylake = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_skylake))]

intel_skylake = {}
intel_skylake['idle_watts'] = (servers_intel_skylake['avg. watts @ active idle'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['100% watts'] = (servers_intel_skylake['avg. watts @ 100%'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['GB/Chip'] = (servers_intel_skylake['Total Memory (GB)'] / servers_intel_skylake['Chips']).mean()
intel_skylake

{'idle_watts': 0.6385993209281899,
 '100% watts': 4.150080573728887,
 'GB/Chip': 80.78571428571429}

### Intel: Cascade Lake

In [9]:
servers_intel_cascadelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_cascadelake))]

intel_cascadelake = {}
intel_cascadelake['idle_watts'] = (servers_intel_cascadelake['avg. watts @ active idle'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['100% watts'] = (servers_intel_cascadelake['avg. watts @ 100%'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['GB/Chip'] = (servers_intel_cascadelake['Total Memory (GB)'] / servers_intel_cascadelake['Chips']).mean()
intel_cascadelake

{'idle_watts': 0.6349836281915363,
 '100% watts': 3.957907252186588,
 'GB/Chip': 97.37142857142857}

### Intel: Coffee Lake

In [10]:
servers_intel_coffeelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_intel_coffeelake))]

intel_coffeelake = {}
intel_coffeelake['idle_watts'] = (servers_intel_coffeelake['avg. watts @ active idle'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['100% watts'] = (servers_intel_coffeelake['avg. watts @ 100%'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['GB/Chip'] = (servers_intel_coffeelake['Total Memory (GB)'] / servers_intel_coffeelake['Chips']).mean()
intel_coffeelake

{'idle_watts': 1.138425925925926,
 '100% watts': 5.421759259259258,
 'GB/Chip': 19.555555555555557}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2bb85156-92ac-4c34-9a24-94671c8f593c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>